This file generates reults for the self-storage dataset.

We first need to process the dataset a little and weed out some columns. Then, we can choose to append whatever embeddings we have extracted previously.

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from mosaiks import config as c
from mosaiks.utils.imports import *
import io as b_io
import numpy as np

In [56]:
# Credit for this cell goes to Scott Fleming, our PhD mentor. He demonstrated which columns to ignore.

cols_to_drop = [
    "access_to_dump_and_wash_station", # Too few facilities
    "bluetooth_enabled_lock", # Only present for certain franchises (e.g. ExtraSpace)
    "enclosed_storage", # Only in ExtraSpace
    "enclosed_vehicle_storage", # Only in ExtraSpace
    "multidoor", # Only in ExtraSpace
    "power_outlet", # Only in ExtraSpace
    "premium_location", # Only in SmartStop
    "rv_parking_uncovered", # Only in ExtraSpace
    "stair_access", # Only in ExtraSpace
    "domain", # A string
    "website", # A string
#     "places_id", # A string
    "address", # A string
    "city", # A string
    "state", # A string
    "country", # A string
    "dimensions" # A string
]

data_path = c.data_dir + "/raw/"

print(data_path)

train_df = pd.read_csv(data_path + "train_df.tsv", sep="\t")
test_id_df = pd.read_csv(data_path + "test_id_df.tsv", sep="\t")
test_ood_df = pd.read_csv(data_path + "test_ood_df.tsv", sep="\t")

for df in [train_df, test_id_df, test_ood_df]:
    df.drop(cols_to_drop, axis=1, inplace=True)

display(train_df)

/home/ubuntu/cs230/mosaiks-paper/data/raw/


,price,places_id,dim1,dim2,square_feet,latitude,longitude,1st_floor_access,24_hr_access,climate_controlled,drive_up_access,elevator_access,inside_unit,outside_unit,oversized_doors,rv_parking,parking,rv_parking_covered,unusual_size,wine_storage
0,72.0,ChIJkxtdA8mzQIYROlwVeGwwh6I,5.0,5.0,25.0,29.979650,-95.275352,0,0,1,0,0,0,0,0,0,0,0,0,0
1,77.0,ChIJkxtdA8mzQIYROlwVeGwwh6I,5.0,5.0,25.0,29.979650,-95.275352,1,0,1,0,0,0,0,0,0,0,0,0,0
2,105.0,ChIJkxtdA8mzQIYROlwVeGwwh6I,5.0,10.0,50.0,29.979650,-95.275352,0,0,1,0,0,0,0,0,0,0,0,0,0
3,119.0,ChIJkxtdA8mzQIYROlwVeGwwh6I,5.0,10.0,50.0,29.979650,-95.275352,1,0,1,0,0,0,0,0,0,0,0,0,0
4,118.0,ChIJkxtdA8mzQIYROlwVeGwwh6I,5.0,15.0,75.0,29.979650,-95.275352,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22096,19.0,ChIJEdlQ-KvV2IcRhNRyDF9stDc,5.0,5.0,25.0,38.597105,-90.585272,0,0,1,0,1,1,0,0,0,0,0,1,0
22097,37.0,ChIJEdlQ-KvV2IcRhNRyDF9stDc,4.0,5.0,20.0,38.597105,-90.585272,1,0,1,0,0,1,0,0,0,0,0,0,0
22098,35.0,ChIJEdlQ-KvV2IcRhNRyDF9stDc,5.0,5.0,25.0,38.597105,-90.585272,0,0,1,0,1,1,0,0,0,0,0,0,0
22099,193.0,ChIJEdlQ-KvV2IcRhNRyDF9stDc,10.0,11.0,110.0,38.597105,-90.585272,1,0,1,0,0,1,0,0,0,0,0,0,0


This cell can load any embeddings except for the "combined" ones. We just call it multitask for now.

In [181]:
with open(c.data_dir + "/int/feature_matrices/" + "resnet18multitask_self_storage.pkl", "rb") as f:
  data = np.load(f, allow_pickle=True)
print(len(data["X"]))
print(type(data["X"]))
print(len(data["latlon"]))
temp = b_io.BytesIO(data["X"])
features = np.load(temp, allow_pickle=True)
print(features.shape)
latlons_samp = data["latlon"]
ids_x = data["ids_X"]
print(ids_x.shape)

X = pd.DataFrame(features, index=ids_x)
print(X.shape)
X

4702336
<class 'bytes'>
2296
(2296, 512)
(2296,)
(2296, 512)


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
ChIJkxtdA8mzQIYROlwVeGwwh6I,0.190669,0.054313,0.082671,0.049968,0.029129,0.015893,0.101152,0.021175,0.044239,0.059315,...,0.010210,0.040955,0.006781,0.090624,0.053648,0.037641,0.066242,0.079581,0.001939,0.070641
ChIJBXG-W2DBIogRs552_nWUOws,0.041142,0.012070,0.035584,0.043557,0.025594,0.019376,0.005663,0.020075,0.021325,0.036361,...,0.029408,0.033453,0.055427,0.015650,0.044909,0.038210,0.067744,0.020030,0.036052,0.001792
ChIJJ6ZCoiSCRoYRjFNyzETgjMc,0.038909,0.056029,0.303082,0.068597,0.022067,0.037382,0.121365,0.024217,0.014294,0.057518,...,0.128624,0.061301,0.114033,0.176825,0.063252,0.081136,0.046816,0.066473,0.197458,0.141041
ChIJGxPXPm9TrIkR8DTsN0sjT24,0.041666,0.016534,0.117750,0.013716,0.037175,0.009686,0.016022,0.020974,0.031051,0.008444,...,0.065990,0.047300,0.062622,0.010161,0.071395,0.027595,0.140282,0.025538,0.031109,0.014150
ChIJQe7mgp2OyIkRyP-yL8dVN00,0.018243,0.084911,0.109830,0.049871,0.096572,0.086286,0.119836,0.032699,0.055218,0.025993,...,0.044033,0.003690,0.171767,0.066344,0.071764,0.037866,0.053400,0.066234,0.063689,0.017967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJRR3QlPfX54kRk_-jdB-UnJk,0.000000,0.020736,0.009786,0.009596,0.016956,0.070474,0.024832,0.014315,0.002462,0.009776,...,0.025229,0.029215,0.003060,0.000020,0.018112,0.023908,0.005949,0.003751,0.005936,0.006791
ChIJ176SKpF744kRqd4b-EaZuW4,0.000000,0.054015,0.005044,0.033401,0.014934,0.006766,0.074103,0.093412,0.004261,0.000698,...,0.002147,0.022620,0.007439,0.003479,0.024751,0.003357,0.030648,0.014618,0.000982,0.028524
ChIJQa049JwR3okRPMVp5oeCFns,0.121545,0.034870,0.059036,0.024356,0.001846,0.007529,0.087827,0.000000,0.030111,0.001062,...,0.064508,0.013800,0.000293,0.082247,0.001041,0.009770,0.034102,0.000000,0.075456,0.002479
ChIJr_-EegO354kRitPgb5lyFmo,0.025808,0.000000,0.019920,0.023024,0.007586,0.007125,0.036461,0.026979,0.010685,0.003847,...,0.018991,0.009193,0.017905,0.013158,0.005992,0.034737,0.000187,0.001173,0.054437,0.023010


This cell loads the "combined" resnet features.

In [95]:
tasks_UAR = ["treecover", "elevation", "population",]
tasks_POP = ["nightlights", "income", "roads", "housing",]

with open(c.data_dir + "/int/feature_matrices/" + "resnet18_combined_POP.pkl", "rb") as f:
  data = np.load(f, allow_pickle=True)
X_list = []
for task in tasks_UAR + tasks_POP:
    if task == "nightlights":
        continue
    print(len(data[f"X_{task}"]["X"]))
    print(type(data[f"X_{task}"]["X"]))
    print(len(data["latlon"]))
    temp = b_io.BytesIO(data[f"X_{task}"]["X"])
    temp2 = np.load(temp, allow_pickle=True)
    X_list.append(temp2)
#     features = data["X"].astype(np.float64)
features = np.concatenate(X_list, axis=1)
print(features.shape)
# ids_x = data["ids_X"].astype('str')

X = pd.DataFrame(features, index=ids_x)

display(X)

204800128
<class 'bytes'>
100000
204800128
<class 'bytes'>
100000
204800128
<class 'bytes'>
100000
204800128
<class 'bytes'>
100000
204800128
<class 'bytes'>
100000
204800128
<class 'bytes'>
100000
(100000, 3072)


,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
"342,2056",0.005640,0.792927,1.617126,0.352360,0.439311,0.728813,1.040530,0.525164,0.496292,0.043982,...,0.196096,0.423205,0.184330,0.284334,0.081031,0.304330,0.179038,0.135682,0.177315,0.297331
"1616,2768",0.000000,0.772168,1.395226,0.209779,0.449780,0.752864,0.791526,0.698561,0.484218,0.088787,...,0.323435,0.497787,0.162754,0.272290,0.131660,0.324005,0.239287,0.152983,0.142142,0.327456
"1009,3643",0.000000,0.681406,1.536598,0.219374,0.212445,1.098867,0.407930,0.619210,0.323425,0.024059,...,0.323918,0.408603,0.161777,0.351066,0.100113,0.347688,0.216257,0.139806,0.196364,0.304534
"1856,2551",0.008329,0.551459,1.110244,0.437601,0.288291,1.113418,0.855563,0.664780,0.557069,0.087785,...,0.244509,0.424838,0.127037,0.331192,0.067270,0.330514,0.203470,0.104971,0.179793,0.399473
"859,3766",0.000000,0.523367,1.473866,0.270569,0.239324,1.098173,0.398499,0.474034,0.288777,0.074651,...,0.324344,0.396497,0.134459,0.308905,0.150054,0.320109,0.197009,0.149296,0.211439,0.316732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"1950,2167",0.000000,0.606742,1.427575,0.231361,0.266070,1.046433,0.630712,0.579037,0.503640,0.066601,...,0.348051,0.418798,0.161957,0.374841,0.111241,0.398328,0.215814,0.184689,0.174367,0.384044
"897,3153",0.036198,0.767727,1.852696,0.260753,0.596597,0.714549,1.250455,0.524793,0.482935,0.226429,...,0.205812,0.552503,0.132834,0.285058,0.156494,0.281956,0.194739,0.074312,0.226212,0.396038
"2257,1998",0.068395,0.500607,1.712002,0.234047,0.501846,0.691533,1.322746,0.450303,0.522651,0.044559,...,0.306056,0.550919,0.140928,0.317614,0.183274,0.284699,0.261136,0.109023,0.229948,0.387431
"304,214",0.000000,0.777098,1.630451,0.324399,0.240140,1.236116,0.620762,0.535121,0.450028,0.061349,...,0.251450,0.516818,0.134119,0.229875,0.066273,0.278815,0.216177,0.137316,0.199602,0.369184


Here, we merge the unit features with embeddings.

In [182]:
train_df_with_features = pd.merge(X, train_df, left_index=True, right_on='places_id', how='right')
display(train_df_with_features)

test_id_df_with_features = pd.merge(X, test_id_df, left_index=True, right_on='places_id', how='right')
display(test_id_df_with_features)

test_ood_df_with_features = pd.merge(X, test_ood_df, left_index=True, right_on='places_id', how='right')
display(test_ood_df_with_features)

,0,1,2,3,4,5,6,7,8,9,...,drive_up_access,elevator_access,inside_unit,outside_unit,oversized_doors,rv_parking,parking,rv_parking_covered,unusual_size,wine_storage
0,0.190669,0.054313,0.082671,0.049968,0.029129,0.015893,0.101152,0.021175,0.044239,0.059315,...,0,0,0,0,0,0,0,0,0,0
1,0.190669,0.054313,0.082671,0.049968,0.029129,0.015893,0.101152,0.021175,0.044239,0.059315,...,0,0,0,0,0,0,0,0,0,0
2,0.190669,0.054313,0.082671,0.049968,0.029129,0.015893,0.101152,0.021175,0.044239,0.059315,...,0,0,0,0,0,0,0,0,0,0
3,0.190669,0.054313,0.082671,0.049968,0.029129,0.015893,0.101152,0.021175,0.044239,0.059315,...,0,0,0,0,0,0,0,0,0,0
4,0.190669,0.054313,0.082671,0.049968,0.029129,0.015893,0.101152,0.021175,0.044239,0.059315,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22096,0.000000,0.023385,0.020241,0.073794,0.009437,0.003446,0.025026,0.042117,0.010865,0.052239,...,0,1,1,0,0,0,0,0,1,0
22097,0.000000,0.023385,0.020241,0.073794,0.009437,0.003446,0.025026,0.042117,0.010865,0.052239,...,0,0,1,0,0,0,0,0,0,0
22098,0.000000,0.023385,0.020241,0.073794,0.009437,0.003446,0.025026,0.042117,0.010865,0.052239,...,0,1,1,0,0,0,0,0,0,0
22099,0.000000,0.023385,0.020241,0.073794,0.009437,0.003446,0.025026,0.042117,0.010865,0.052239,...,0,0,1,0,0,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9,...,drive_up_access,elevator_access,inside_unit,outside_unit,oversized_doors,rv_parking,parking,rv_parking_covered,unusual_size,wine_storage
0,0.043107,0.017948,0.082264,0.036021,0.000819,0.026163,0.055578,0.022084,0.009869,0.039602,...,0,0,1,0,0,0,0,0,0,0
1,0.043107,0.017948,0.082264,0.036021,0.000819,0.026163,0.055578,0.022084,0.009869,0.039602,...,1,0,0,1,0,0,0,0,0,0
2,0.043107,0.017948,0.082264,0.036021,0.000819,0.026163,0.055578,0.022084,0.009869,0.039602,...,0,0,1,0,0,0,0,0,0,0
3,0.043107,0.017948,0.082264,0.036021,0.000819,0.026163,0.055578,0.022084,0.009869,0.039602,...,0,0,1,0,0,0,0,0,0,0
4,0.043107,0.017948,0.082264,0.036021,0.000819,0.026163,0.055578,0.022084,0.009869,0.039602,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5550,0.026429,0.048848,0.029667,0.033995,0.038089,0.025152,0.081532,0.028562,0.024667,0.024963,...,0,0,1,0,0,0,0,0,0,0
5551,0.026429,0.048848,0.029667,0.033995,0.038089,0.025152,0.081532,0.028562,0.024667,0.024963,...,1,0,0,0,0,0,0,0,0,0
5552,0.026429,0.048848,0.029667,0.033995,0.038089,0.025152,0.081532,0.028562,0.024667,0.024963,...,1,0,0,0,0,0,0,0,0,0
5553,0.026429,0.048848,0.029667,0.033995,0.038089,0.025152,0.081532,0.028562,0.024667,0.024963,...,1,0,0,0,0,0,0,0,0,0


,0,1,2,3,4,5,6,7,8,9,...,drive_up_access,elevator_access,inside_unit,outside_unit,oversized_doors,rv_parking,parking,rv_parking_covered,unusual_size,wine_storage
0,0.035781,0.020297,0.079792,0.012490,0.053654,0.023053,0.026158,0.005055,0.019553,0.023674,...,0,0,1,0,0,0,0,0,0,0
1,0.035781,0.020297,0.079792,0.012490,0.053654,0.023053,0.026158,0.005055,0.019553,0.023674,...,0,1,1,0,0,0,0,0,0,0
2,0.035781,0.020297,0.079792,0.012490,0.053654,0.023053,0.026158,0.005055,0.019553,0.023674,...,0,0,1,0,0,0,0,0,0,0
3,0.035781,0.020297,0.079792,0.012490,0.053654,0.023053,0.026158,0.005055,0.019553,0.023674,...,0,0,1,0,0,0,0,0,0,0
4,0.035781,0.020297,0.079792,0.012490,0.053654,0.023053,0.026158,0.005055,0.019553,0.023674,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5424,0.157866,0.052295,0.006933,0.013647,0.015301,0.004428,0.076627,0.003957,0.012040,0.006794,...,1,0,0,0,0,0,0,0,0,0
5425,0.157866,0.052295,0.006933,0.013647,0.015301,0.004428,0.076627,0.003957,0.012040,0.006794,...,1,0,0,0,0,0,0,0,0,0
5426,0.157866,0.052295,0.006933,0.013647,0.015301,0.004428,0.076627,0.003957,0.012040,0.006794,...,1,0,0,0,0,0,0,0,0,0
5427,0.157866,0.052295,0.006933,0.013647,0.015301,0.004428,0.076627,0.003957,0.012040,0.006794,...,1,0,0,0,0,0,0,0,0,0


### Let's train ridge regression

In [183]:
solver = solve.ridge_regression

lambdas = [1e-2, 1e-1, 1e0, 1e1, 1e2]

solver_kwargs = {
    "lambdas": lambdas,
    "return_preds": True,
    "svd_solve": False,
    "clip_bounds": None,
}

In [184]:
# no features added
this_X = train_df.drop("price", axis=1).drop("places_id", axis=1)
this_Y = train_df["price"]
this_X_test = test_id_df.drop("price", axis=1).drop("places_id", axis=1)
this_Y_test = test_id_df["price"]
this_X_test_ood = test_ood_df.drop("price", axis=1).drop("places_id", axis=1)
this_Y_test_ood = test_ood_df["price"]

In [185]:
# remove _with_features
this_X_with_features = train_df_with_features.drop("price", axis=1).drop("places_id", axis=1)
this_Y = train_df["price"]
this_X_test_with_features = test_id_df_with_features.drop("price", axis=1).drop("places_id", axis=1)
this_Y_test = test_id_df["price"]
this_X_test_ood_with_features = test_ood_df_with_features.drop("price", axis=1).drop("places_id", axis=1)
this_Y_test_ood = test_ood_df["price"]

In [186]:
print("Training model...")
kfold_results = solve.kfold_solve(
    this_X_with_features,
    this_Y,
    solve_function=solver,
    num_folds=5,
    return_model=True,
    **solver_kwargs
)
print("")

best_lambda_idx, best_metrics, best_preds = ir.interpret_kfold_results(
    kfold_results, "r2_score", hps=[("lambdas", solver_kwargs["lambdas"])]
)
best_lambdas = np.array(
    [solver_kwargs["lambdas"][np.asarray(best_lambda_idx).squeeze()]]
)

Training model...
on fold (of 5): 1 2 3 4 5 


/home/ubuntu/cs230/mosaiks-paper/code/mosaiks/solve/solve_functions.py:364: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  "y_true_test": np.array(kfold_y_test),
/home/ubuntu/cs230/mosaiks-paper/code/mosaiks/solve/solve_functions.py:365: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  "y_true_train": np.array(kfold_y_train),


### The following are different tests using different embeddings.

In [203]:
holdout_results = solve.single_solve(
    this_X,
    this_X_test,
    this_Y,
    this_Y_test,
    lambdas=best_lambdas,
    return_preds=True,
    return_model=False,
    clip_bounds=None,
    svd_solve=False,
)
print(holdout_results)

{'metrics_test': array([[[{'mse': 9641.921063206488, 'r2_score': 0.612293420341437}]]],
      dtype=object), 'metrics_train': array([[[{'mse': 22405.921920695222, 'r2_score': 0.5619572089857816}]]],
      dtype=object), 'y_true_test': array([ 93.,  98.,  99., ..., 177.,  97., 254.]), 'y_true_train': array([ 72.,  77., 105., ...,  35., 193.,  61.]), 'hp_warning': array([[[False]]], dtype=object), 'y_pred_test': array([[[array([ 61.229595  ,  60.04069889, 111.72883992, ..., 148.40090747,
                 86.86536638, 259.85192215])                                ]]],
      dtype=object), 'y_pred_train': array([[[array([ 78.75353859,  55.64729616, 129.25278351, ...,  49.62999831,
                189.71671686, 100.12924323])                                ]]],
      dtype=object)}


In [204]:
holdout_results = solve.single_solve(
    this_X,
    this_X_test_ood,
    this_Y,
    this_Y_test_ood,
    lambdas=best_lambdas,
    return_preds=True,
    return_model=False,
    clip_bounds=None,
    svd_solve=False,
)
print(holdout_results)

{'metrics_test': array([[[{'mse': 15692.677666891508, 'r2_score': 0.5925148189544338}]]],
      dtype=object), 'metrics_train': array([[[{'mse': 22405.921920695222, 'r2_score': 0.5619572089857816}]]],
      dtype=object), 'y_true_test': array([ 87.,  91., 114., ..., 225., 270., 296.]), 'y_true_train': array([ 72.,  77., 105., ...,  35., 193.,  61.]), 'hp_warning': array([[[False]]], dtype=object), 'y_pred_test': array([[[array([ 93.78512275,  53.77204924,  86.02144706, ..., 259.46791387,
                315.19342121, 370.91892855])                                ]]],
      dtype=object), 'y_pred_train': array([[[array([ 78.75353859,  55.64729616, 129.25278351, ...,  49.62999831,
                189.71671686, 100.12924323])                                ]]],
      dtype=object)}


In [205]:
holdout_results = solve.single_solve(
    this_X_with_features,
    this_X_test_ood_with_features,
    this_Y,
    this_Y_test_ood,
    lambdas=best_lambdas,
    return_preds=True,
    return_model=False,
    clip_bounds=None,
    svd_solve=False,
)
print(holdout_results)

{'metrics_test': array([[[{'mse': 17904.26456148936, 'r2_score': 0.5350874693795082}]]],
      dtype=object), 'metrics_train': array([[[{'mse': 20261.61382306693, 'r2_score': 0.6038791038850003}]]],
      dtype=object), 'y_true_test': array([ 87.,  91., 114., ..., 225., 270., 296.]), 'y_true_train': array([ 72.,  77., 105., ...,  35., 193.,  61.]), 'hp_warning': array([[[False]]], dtype=object), 'y_pred_test': array([[[array([ 50.94274403,   3.81174481,  56.28492488, ..., 246.71140825,
                303.05104688, 359.39068552])                                ]]],
      dtype=object), 'y_pred_train': array([[[array([ 56.67147167,  33.18210436, 107.86253156, ...,   5.68097528,
                152.61204323,  56.87203518])                                ]]],
      dtype=object)}


In [206]:
holdout_results = solve.single_solve(
    this_X_with_features,
    this_X_test_with_features,
    this_Y,
    this_Y_test,
    lambdas=best_lambdas,
    return_preds=True,
    return_model=False,
    clip_bounds=None,
    svd_solve=False,
)
print(holdout_results)

{'metrics_test': array([[[{'mse': 11076.206030158892, 'r2_score': 0.5546200879061802}]]],
      dtype=object), 'metrics_train': array([[[{'mse': 20261.61382306693, 'r2_score': 0.6038791038850003}]]],
      dtype=object), 'y_true_test': array([ 93.,  98.,  99., ..., 177.,  97., 254.]), 'y_true_train': array([ 72.,  77., 105., ...,  35., 193.,  61.]), 'hp_warning': array([[[False]]], dtype=object), 'y_pred_test': array([[[array([-13.00319369, -11.43471051,  38.1878662 , ..., 131.06140095,
                 70.23095253, 243.74067823])                                ]]],
      dtype=object), 'y_pred_train': array([[[array([ 56.67147167,  33.18210436, 107.86253156, ...,   5.68097528,
                152.61204323,  56.87203518])                                ]]],
      dtype=object)}
